<a href="https://colab.research.google.com/github/betty9895/ml-0716/blob/master/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload() #upload kaggle.json

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

Saving kaggle.json to kaggle (1).json


In [ ]:
!kaggle competitions download -c titanic

test.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import pandas as pd

train_df = pd.read_csv("train.csv", encoding="utf8")
test_df = pd.read_csv("test.csv", encoding="utf8")
# train_pd

In [ ]:
datas = pd.concat([train_df, test_df])
datas = datas.drop(["PassengerId","Survived"], axis=1)
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
# step 1 補缺失值  (補最可能)
s = datas.isna().sum()
# 篩選 [[True]]
s[s>0]

Age          263
Fare           1
Cabin       1014
Embarked       2
dtype: int64

In [ ]:
most = datas["Embarked"].value_counts().idxmax()
datas["Embarked"] = datas["Embarked"].fillna(most)

In [ ]:
# 補數字類型[中位數]
med = datas.median().drop(["Pclass"])
datas = datas.fillna(med)
datas.isna().sum()

Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin       1014
Embarked       0
dtype: int64

In [ ]:
# name ticket
# 先準備要保留的list
def name(n):
  whitelist = []
  n = n.split(",")[-1].split(".")[0]
  return n.strip()
v = datas["Name"].apply(name).value_counts()
white = v.index[:4]

In [ ]:
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,"Spector, Mr. Woolf",male,28.0,0,0,A.5. 3236,8.0500,NaN,S
414,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,"Ware, Mr. Frederick",male,28.0,0,0,359309,8.0500,NaN,S


In [ ]:
def name(n):
    n = n.split(",")[-1].split(".")[0]
    n = n.strip()
    if n in white:
        return n
    else:
        return None
datas["Name"] = datas["Name"].apply(name)
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,Miss,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,Mrs,female,35.0,1,0,113803,53.1000,C123,S
4,3,Mr,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,Mr,male,28.0,0,0,A.5. 3236,8.0500,NaN,S
414,1,None,female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,Mr,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,Mr,male,28.0,0,0,359309,8.0500,NaN,S


In [ ]:
v = datas["Ticket"].value_counts()
def ticket(t):
  global v
  return v[t]
datas["Ticket"] = datas["Ticket"].apply(ticket)

In [ ]:
datas = datas.drop(["Cabin"], axis=1)

In [ ]:
# 如果你想要重製index datas.reset_index(drop=True)
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,Mr,male,22.0,1,0,1,7.2500,S
1,1,Mrs,female,38.0,1,0,2,71.2833,C
2,3,Miss,female,26.0,0,0,1,7.9250,S
3,1,Mrs,female,35.0,1,0,2,53.1000,S
4,3,Mr,male,35.0,0,0,1,8.0500,S
...,...,...,...,...,...,...,...,...,...
413,3,Mr,male,28.0,0,0,1,8.0500,S
414,1,None,female,39.0,0,0,3,108.9000,C
415,3,Mr,male,38.5,0,0,1,7.2500,S
416,3,Mr,male,28.0,0,0,1,8.0500,S


In [ ]:
# one-hot encoding  類別處理
datas = pd.get_dummies(datas)
datas

,Pclass,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,1
1,1,38.0,1,0,2,71.2833,0,0,0,1,1,0,1,0,0
2,3,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,1
3,1,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1
4,3,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,3,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,1
414,1,39.0,0,0,3,108.9000,0,0,0,0,1,0,1,0,0
415,3,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,1
416,3,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,1


In [ ]:
# 取列：iloc(看實際第幾列)、loc(看列標籤)
# [第一列,第二列,第三列]
x_train = datas.iloc[:len(train_df)]
x_predict = datas.iloc[len(train_df):]
y_train = train_df["Survived"]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
clf = RandomForestClassifier(random_state=0)
params = {
    "n_estimators":range(20, 100, 5),
    "max_depth":range(5, 10)
}
search = GridSearchCV(clf, params, cv = 10, n_jobs=-1)
search.fit(x_train, y_train)
print(search.best_params_)
print(search.best_score_)

{'max_depth': 7, 'n_estimators': 70}
0.8373033707865168


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np
clf = RandomForestClassifier(n_estimators=70, max_depth=7, random_state=0)
scores = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=1)
print(scores)
print(np.average(scores))

[0.8        0.88764045 0.76404494 0.8988764  0.85393258 0.82022472
 0.84269663 0.78651685 0.87640449 0.84269663]
0.8373033707865168


In [ ]:
clf = RandomForestClassifier(n_estimators=70, max_depth=7)
clf.fit(x_train, y_train)
pre = clf.predict(x_predict)
df = pd.DataFrame({
    "PassengerId":test_df["PassengerId"],
    "Survived":pre
})
df.to_csv("titanic_rf.csv", encoding="utf-8", index=False)
df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
# !kaggle competitions submit -c titanic -f titanic_rf.csv -m "Message"

100% 2.77k/2.77k [00:00<00:00, 13.2kB/s]
Successfully submitted to Titanic: Machine Learning from Disaster